# FizzBuzz

FizzBuzz是一个简单的小游戏。游戏规则如下：从1开始往上数数，当遇到3的倍数的时候，说fizz，当遇到5的倍数，说buzz，当遇到15的倍数，就说fizzbuzz，其他情况下则正常数数。

我们可以写一个简单的小程序来决定要返回正常数值还是fizz, buzz 或者 fizzbuzz。

In [1]:
def fizz_buzz_encode(i):
    if i % 15 == 0: return 3
    elif i % 5 == 0: return 2
    elif i % 3 == 0: return 1
    else: return 0
def fizz_buzz_decode(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]
    
def helper(i):
    print(fizz_buzz_decode(i, fizz_buzz_encode(i)))

for i in range(1,16):
    helper(i)

1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz


我们首先定义模型的输入与输出(训练数据)

In [2]:
import numpy as np
import torch

NUM_DIGITS = 10

def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)][::-1])
trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = torch.LongTensor([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])


然后我们用PyTorch定义模型

In [9]:
NUM_HIDDEN = 100
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
)
if torch.cuda.is_available():
    model = model.cuda()

- 为了让我们的模型学会FizzBuzz这个游戏，我们需要定义一个损失函数，和一个优化算法。
- 这个优化算法会不断优化（降低）损失函数，使得模型的在该任务上取得尽可能低的损失值。
- 损失值低往往表示我们的模型表现好，损失值高表示我们的模型表现差。
- 由于FizzBuzz游戏本质上是一个分类问题，我们选用Cross Entropyy Loss函数。
- 优化函数我们选用Stochastic Gradient Descent。

In [10]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [11]:
BATCH_SIZE = 128

for start in range(0, len(trX), BATCH_SIZE):
    end = start + BATCH_SIZE
    print(start,end)
    batchX = trX[start:end]
    batchY = trY[start:end]
print(trX)
print(trY)

0 128
128 256
256 384
384 512
512 640
640 768
768 896
896 1024
tensor([[0., 0., 0.,  ..., 1., 0., 1.],
        [0., 0., 0.,  ..., 1., 1., 0.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 0., 1.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]])
tensor([0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0,
        2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0,
        0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0,
        0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0,
        0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2,
        0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0,
        2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0,
        0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0,
        0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0,

以下是模型的训练代码

In [12]:
BATCH_SIZE = 128
for epoch in range(10000):
     for start in range(0, len(trX), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = trX[start:end]
        batchY = trY[start:end]
        
        if torch.cuda.is_available():
            batchX = batchX.cuda()
            batchY = batchY.cuda()
        
        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)
        
        print("Epoch", epoch, loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
            

Epoch 0 1.4624881744384766
Epoch 0 1.4316463470458984
Epoch 0 1.4441479444503784
Epoch 0 1.479253888130188
Epoch 0 1.5144762992858887
Epoch 0 1.4839832782745361
Epoch 0 1.4981836080551147
Epoch 0 1.496993899345398
Epoch 1 1.4489264488220215
Epoch 1 1.417574167251587
Epoch 1 1.4289705753326416
Epoch 1 1.4645377397537231
Epoch 1 1.4986225366592407
Epoch 1 1.4667879343032837
Epoch 1 1.4806448221206665
Epoch 1 1.4757345914840698
Epoch 2 1.4361199140548706
Epoch 2 1.404223918914795
Epoch 2 1.4145171642303467
Epoch 2 1.4504414796829224
Epoch 2 1.483384609222412
Epoch 2 1.450264811515808
Epoch 2 1.4638200998306274
Epoch 2 1.4553292989730835
Epoch 3 1.4237717390060425
Epoch 3 1.3914103507995605
Epoch 3 1.4006620645523071
Epoch 3 1.436901330947876
Epoch 3 1.4687005281448364
Epoch 3 1.4344067573547363
Epoch 3 1.4477322101593018
Epoch 3 1.4358487129211426
Epoch 4 1.4118731021881104
Epoch 4 1.379113793373108
Epoch 4 1.3873859643936157
Epoch 4 1.4239020347595215
Epoch 4 1.454582929611206
Epoch 4 1.

Epoch 49 1.1686761379241943
Epoch 49 1.1583678722381592
Epoch 49 1.150510311126709
Epoch 49 1.1767789125442505
Epoch 49 1.1719847917556763
Epoch 49 1.1422525644302368
Epoch 49 1.1640441417694092
Epoch 49 1.1087042093276978
Epoch 50 1.1673179864883423
Epoch 50 1.1575260162353516
Epoch 50 1.1496785879135132
Epoch 50 1.1756083965301514
Epoch 50 1.1705259084701538
Epoch 50 1.1412098407745361
Epoch 50 1.1632941961288452
Epoch 50 1.1080403327941895
Epoch 51 1.1660401821136475
Epoch 51 1.1567424535751343
Epoch 51 1.1489094495773315
Epoch 51 1.1745140552520752
Epoch 51 1.1691628694534302
Epoch 51 1.1402500867843628
Epoch 51 1.1626237630844116
Epoch 51 1.1074391603469849
Epoch 52 1.1648383140563965
Epoch 52 1.1560131311416626
Epoch 52 1.1481952667236328
Epoch 52 1.1734894514083862
Epoch 52 1.1678889989852905
Epoch 52 1.1393661499023438
Epoch 52 1.162017583847046
Epoch 52 1.1069035530090332
Epoch 53 1.1637059450149536
Epoch 53 1.1553317308425903
Epoch 53 1.1475319862365723
Epoch 53 1.17252957820

Epoch 93 1.1283349990844727
Epoch 93 1.1545660495758057
Epoch 93 1.098936676979065
Epoch 94 1.1445772647857666
Epoch 94 1.1433371305465698
Epoch 94 1.1369357109069824
Epoch 94 1.156628131866455
Epoch 94 1.1479852199554443
Epoch 94 1.1282631158828735
Epoch 94 1.1544978618621826
Epoch 94 1.098730444908142
Epoch 95 1.1443769931793213
Epoch 95 1.1431918144226074
Epoch 95 1.1368136405944824
Epoch 95 1.1564594507217407
Epoch 95 1.147802472114563
Epoch 95 1.1281956434249878
Epoch 95 1.154431700706482
Epoch 95 1.0985236167907715
Epoch 96 1.144179105758667
Epoch 96 1.1430494785308838
Epoch 96 1.136695384979248
Epoch 96 1.1562941074371338
Epoch 96 1.1476247310638428
Epoch 96 1.1281276941299438
Epoch 96 1.1543680429458618
Epoch 96 1.0983129739761353
Epoch 97 1.1439889669418335
Epoch 97 1.142909049987793
Epoch 97 1.1365793943405151
Epoch 97 1.156133770942688
Epoch 97 1.1474478244781494
Epoch 97 1.1280601024627686
Epoch 97 1.1543058156967163
Epoch 97 1.0981042385101318
Epoch 98 1.1438031196594238
E

Epoch 129 1.12630295753479
Epoch 129 1.1526155471801758
Epoch 129 1.0909178256988525
Epoch 130 1.1390109062194824
Epoch 130 1.1390860080718994
Epoch 130 1.1333777904510498
Epoch 130 1.1518820524215698
Epoch 130 1.1427911520004272
Epoch 130 1.1262519359588623
Epoch 130 1.1525640487670898
Epoch 130 1.0906801223754883
Epoch 131 1.1388747692108154
Epoch 131 1.1389833688735962
Epoch 131 1.1332919597625732
Epoch 131 1.1517763137817383
Epoch 131 1.142667293548584
Epoch 131 1.1262054443359375
Epoch 131 1.1525150537490845
Epoch 131 1.0904462337493896
Epoch 132 1.1387391090393066
Epoch 132 1.1388815641403198
Epoch 132 1.1332062482833862
Epoch 132 1.1516737937927246
Epoch 132 1.1425471305847168
Epoch 132 1.126155138015747
Epoch 132 1.1524648666381836
Epoch 132 1.0901985168457031
Epoch 133 1.1386085748672485
Epoch 133 1.138780951499939
Epoch 133 1.133118987083435
Epoch 133 1.1515721082687378
Epoch 133 1.142429232597351
Epoch 133 1.126105546951294
Epoch 133 1.1524062156677246
Epoch 133 1.0899616479

Epoch 176 1.0791922807693481
Epoch 177 1.1325551271438599
Epoch 177 1.1345587968826294
Epoch 177 1.1293063163757324
Epoch 177 1.1474168300628662
Epoch 177 1.1373043060302734
Epoch 177 1.1241111755371094
Epoch 177 1.1502745151519775
Epoch 177 1.0789481401443481
Epoch 178 1.1324102878570557
Epoch 178 1.134464979171753
Epoch 178 1.1292132139205933
Epoch 178 1.1473219394683838
Epoch 178 1.1371893882751465
Epoch 178 1.124058485031128
Epoch 178 1.150232195854187
Epoch 178 1.0787014961242676
Epoch 179 1.1322635412216187
Epoch 179 1.1343684196472168
Epoch 179 1.1291178464889526
Epoch 179 1.1472258567810059
Epoch 179 1.1370806694030762
Epoch 179 1.1240054368972778
Epoch 179 1.1501797437667847
Epoch 179 1.0784538984298706
Epoch 180 1.1321215629577637
Epoch 180 1.1342763900756836
Epoch 180 1.129024624824524
Epoch 180 1.1471281051635742
Epoch 180 1.1369725465774536
Epoch 180 1.1239509582519531
Epoch 180 1.1501301527023315
Epoch 180 1.0782057046890259
Epoch 181 1.1319773197174072
Epoch 181 1.134183

...

Epoch 9853 0.04057776927947998
Epoch 9853 0.03767894208431244
Epoch 9853 0.03219557926058769
Epoch 9853 0.04139593988656998
Epoch 9853 0.05650767683982849
Epoch 9853 0.011526284739375114
Epoch 9854 0.02361697517335415
Epoch 9854 0.043563466519117355
Epoch 9854 0.04057285189628601
Epoch 9854 0.03767457231879234
Epoch 9854 0.03220589831471443
Epoch 9854 0.04137014225125313
Epoch 9854 0.056460972875356674
Epoch 9854 0.01151927188038826
Epoch 9855 0.02360004372894764
Epoch 9855 0.043545786291360855
Epoch 9855 0.04054320976138115
Epoch 9855 0.0376596674323082
Epoch 9855 0.03218269720673561
Epoch 9855 0.04135658219456673
Epoch 9855 0.05644191429018974
Epoch 9855 0.011509984731674194
Epoch 9856 0.023601273074746132
Epoch 9856 0.04352746531367302
Epoch 9856 0.040528781712055206
Epoch 9856 0.03763410821557045
Epoch 9856 0.03217880427837372
Epoch 9856 0.04132577031850815
Epoch 9856 0.056436263024806976
Epoch 9856 0.011516230180859566
Epoch 9857 0.023577654734253883
Epoch 9857 0.04350466281175613

Epoch 9887 0.023264991119503975
Epoch 9887 0.04298228397965431
Epoch 9887 0.040002185851335526
Epoch 9887 0.03716134652495384
Epoch 9887 0.03173360973596573
Epoch 9887 0.040825482457876205
Epoch 9887 0.05565957725048065
Epoch 9887 0.011351474560797215
Epoch 9888 0.02324105054140091
Epoch 9888 0.042951326817274094
Epoch 9888 0.039975401014089584
Epoch 9888 0.037141475826501846
Epoch 9888 0.03173135221004486
Epoch 9888 0.04079263657331467
Epoch 9888 0.055646564811468124
Epoch 9888 0.011349176988005638
Epoch 9889 0.023237360641360283
Epoch 9889 0.042939092963933945
Epoch 9889 0.03997613489627838
Epoch 9889 0.03711763024330139
Epoch 9889 0.03169414401054382
Epoch 9889 0.04078004136681557
Epoch 9889 0.055631235241889954
Epoch 9889 0.011333047412335873
Epoch 9890 0.0232153981924057
Epoch 9890 0.04292021691799164
Epoch 9890 0.039953939616680145
Epoch 9890 0.03711022436618805
Epoch 9890 0.031710416078567505
Epoch 9890 0.04075416177511215
Epoch 9890 0.055606909096241
Epoch 9890 0.01132931746542

Epoch 9921 0.054888080805540085
Epoch 9921 0.011172939091920853
Epoch 9922 0.022875500842928886
Epoch 9922 0.04237139970064163
Epoch 9922 0.03940451517701149
Epoch 9922 0.03661836311221123
Epoch 9922 0.03125360980629921
Epoch 9922 0.04024693742394447
Epoch 9922 0.05486072972416878
Epoch 9922 0.011172120459377766
Epoch 9923 0.02286888286471367
Epoch 9923 0.042356207966804504
Epoch 9923 0.039381157606840134
Epoch 9923 0.03657842427492142
Epoch 9923 0.031237680464982986
Epoch 9923 0.04024333506822586
Epoch 9923 0.05486216023564339
Epoch 9923 0.011160713620483875
Epoch 9924 0.02287566103041172
Epoch 9924 0.04233710467815399
Epoch 9924 0.039365533739328384
Epoch 9924 0.03658275306224823
Epoch 9924 0.031227633357048035
Epoch 9924 0.0402180477976799
Epoch 9924 0.0548180527985096
Epoch 9924 0.01115882582962513
Epoch 9925 0.022841790691018105
Epoch 9925 0.04231323301792145
Epoch 9925 0.03935396298766136
Epoch 9925 0.03655475750565529
Epoch 9925 0.03120937943458557
Epoch 9925 0.04020246863365173

Epoch 9958 0.03965495526790619
Epoch 9958 0.05404103174805641
Epoch 9958 0.01097689475864172
Epoch 9959 0.02249164693057537
Epoch 9959 0.0417386069893837
Epoch 9959 0.038763731718063354
Epoch 9959 0.036033086478710175
Epoch 9959 0.030731434002518654
Epoch 9959 0.03965623676776886
Epoch 9959 0.054029423743486404
Epoch 9959 0.010978571139276028
Epoch 9960 0.022484146058559418
Epoch 9960 0.04172961041331291
Epoch 9960 0.03876602277159691
Epoch 9960 0.03601758927106857
Epoch 9960 0.030735014006495476
Epoch 9960 0.03964002802968025
Epoch 9960 0.05398470163345337
Epoch 9960 0.010977138765156269
Epoch 9961 0.022457139566540718
Epoch 9961 0.041710805147886276
Epoch 9961 0.03875110670924187
Epoch 9961 0.03601635992527008
Epoch 9961 0.030715452507138252
Epoch 9961 0.039636485278606415
Epoch 9961 0.053994644433259964
Epoch 9961 0.010968627408146858
Epoch 9962 0.022460266947746277
Epoch 9962 0.04169348627328873
Epoch 9962 0.038736265152692795
Epoch 9962 0.035972557961940765
Epoch 9962 0.0307010374

Epoch 9996 0.030237719416618347
Epoch 9996 0.03907449543476105
Epoch 9996 0.05319567024707794
Epoch 9996 0.010790540836751461
Epoch 9997 0.022083377465605736
Epoch 9997 0.04110606014728546
Epoch 9997 0.038146231323480606
Epoch 9997 0.03545203432440758
Epoch 9997 0.030234625563025475
Epoch 9997 0.03906233236193657
Epoch 9997 0.05314895510673523
Epoch 9997 0.010789746418595314
Epoch 9998 0.022060828283429146
Epoch 9998 0.04110722988843918
Epoch 9998 0.038134995847940445
Epoch 9998 0.0354357548058033
Epoch 9998 0.03020278550684452
Epoch 9998 0.03906095772981644
Epoch 9998 0.053168732672929764
Epoch 9998 0.01078308466821909
Epoch 9999 0.022074421867728233
Epoch 9999 0.041079647839069366
Epoch 9999 0.0381133109331131
Epoch 9999 0.03540943190455437
Epoch 9999 0.030213551595807076
Epoch 9999 0.03903735801577568
Epoch 9999 0.05310633406043053
Epoch 9999 0.01077891606837511


最后我们用训练好的模型尝试在1到100这些数字上玩FizzBuzz游戏

In [13]:
testX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(1,101)])
if torch.cuda.is_available():
    testX = testX.cuda()
with torch.no_grad():
    testY = model(testX)
# predicts = zip(range(1, 101), list(testY.max(1)[1]))

In [14]:
predictions = zip(range(1, 101), testY.max(1)[1].cpu().data.tolist())
print([fizz_buzz_decode(i, x)for i,x in predictions])

['1', '2', '3', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', 'buzz', '22', '23', 'fizz', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz', '34', '35', 'fizz', '37', '38', 'fizz', '40', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', '50', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', '66', '67', '68', 'buzz', '70', '71', 'fizz', '73', '74', 'fizzbuzz', 'buzz', '77', 'fizz', '79', 'buzz', 'fizz', '82', '83', 'fizz', 'buzz', '86', '87', '88', '89', 'fizzbuzz', '91', '92', 'fizz', '94', 'buzz', 'fizz', '97', '98', '99', '100']
